# Chat de Recrutamento

In [ ]:
import sqlite3
from sentence_transformers import SentenceTransformer, util # Util é da similaridade do cosseno.
# Os transformadores de frases são criados em arquiteturas de transformadores, que se destacam na captura de relacionamentos contextuais em dados sequenciais.
# Os transformadores utilizam mecanismos de autoatenção, permitindo que o modelo pondere diferentes partes do texto de entrada ao criar embeddings.


### Dados

In [ ]:
# Visualiza o banco de dados
conn = sqlite3.connect("recrutamento.db")
cursor = conn.cursor()
# Um cursor de banco de dados é uma estrutura de controle que permite a um aplicativo navegar e processar linhas de dados em um conjunto de resultados,
# uma de cada vez.

cursor.execute("SELECT * FROM faq LIMIT 5;")
# fetchall() é um método usado para buscar todas as linhas restantes de um resultado de consulta de um banco de dados
resultados = cursor.fetchall()

conn.close()

print(f"\nPrimeiras 5 linhas da tabela faq':\n")
for linha in resultados:
    print(linha)


Primeiras 5 linhas da tabela faq':

(1, 'Como posso me candidatar a uma vaga?', 'Você pode se candidatar acessando nosso site oficial e preenchendo o formulário na área de Carreiras.')
(2, 'Quanto tempo leva para eu receber uma resposta após me candidatar?', 'Normalmente, entramos em contato dentro de 7 a 10 dias úteis, mas isso pode variar conforme o volume de vagas.')
(3, 'Quais são as etapas do processo seletivo?', 'O processo seletivo inclui triagem de currículos, entrevista inicial, entrevista técnica e entrevista final.')
(4, 'Quais benefícios a empresa oferece?', 'Oferecemos assistência médica, vale-alimentação, bonificação, horários flexíveis e home office para algumas funções.')
(5, 'Posso remarcar a data da minha entrevista?', 'Sim, basta entrar em contato com o RH com pelo menos 24 horas de antecedência.')


In [ ]:
# A função do banco de dados.
def carregar_faq_do_banco():
    conn = sqlite3.connect("recrutamento.db")
    cursor = conn.cursor()
  

    cursor.execute("SELECT pergunta, resposta FROM faq")
    faq_data = cursor.fetchall()

    conn.close()

    return faq_data

### Cria Chatbot

In [ ]:
def criar_chatbot():
    model = SentenceTransformer('all-MiniLM-L6-v2') # Cria o modelo

    faq_data = carregar_faq_do_banco() # Carrega o banco

    perguntas_faq = [qa[0] for qa in faq_data]
    respostas_faq = [qa[1] for qa in faq_data]
    # Os embeddings são vetores criados por modelos de aprendizado de máquina com a finalidade de capturar dados significativos sobre cada objeto.
    # perguntas_embeddings são as representações vetoriais numéricas
    # das perguntas .
    perguntas_embeddings = model.encode(perguntas_faq, convert_to_tensor=True) 

    return model, perguntas_faq, respostas_faq, perguntas_embeddings

### Obtêm Respostas

In [ ]:
def obter_resposta(model, perguntas_faq, respostas_faq, perguntas_embeddings, pergunta_usuario):

    emb_usuario = model.encode(pergunta_usuario, convert_to_tensor=True) # Tranforma a pergunta do usuário em um Embeddings.

    similaridades = util.cos_sim(emb_usuario, perguntas_embeddings) # Similaridade entre o embeddings da pergunta e o embeddings do usuário.

    idx_melhor = similaridades[0].argmax().item() # Da similaridade pegamos o índice da melhor pergunta. Argmax() retorna o índice do maior valor do array 
   
    # Retorna a resposta com o melhor índice e a similaridade da pegunta de melhor índice.
    return respostas_faq[idx_melhor], similaridades[0][idx_melhor].item() 

### Mantém Conversa

In [6]:
def conversar_com_chatbot(model, perguntas_faq, respostas_faq, perguntas_embeddings):
    print("Olá! Sou o Chatbot de Recrutamento. Como posso ajudar?\n")

    while True:
        pergunta = input("Você: ")
        if pergunta.lower() in ["sair", "exit", "quit"]:
            print("Chatbot: Obrigado por conversar. Até mais!")
            break

        resposta, score = obter_resposta(
            model,
            perguntas_faq,
            respostas_faq,
            perguntas_embeddings,
            pergunta
        )

        print(f"Chatbot: {resposta}\n(Similaridade: {score:.2f})\n")

### Execução

In [7]:
model, perguntas_faq, respostas_faq, perguntas_embeddings = criar_chatbot()

conversar_com_chatbot(model, perguntas_faq, respostas_faq, perguntas_embeddings)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Olá! Sou o Chatbot de Recrutamento. Como posso ajudar?

Você: Como eu posso me candidatar a uma vaga?
Chatbot: Você pode se candidatar acessando nosso site oficial e preenchendo o formulário na área de Carreiras.
(Similaridade: 0.97)

Você: Posso remarcar minha entrevista?
Chatbot: Sim, basta entrar em contato com o RH com pelo menos 24 horas de antecedência.
(Similaridade: 0.79)

Você: sair
Chatbot: Obrigado por conversar. Até mais!
